# 03. Monitoreo semiautomático de noticias con IA.

Alejandro Villegas
**MODIFICAR**

2025-05-16

Esta proyecto/demostración está realizado en lenguaje Python. Para su construcción se utilizó el entorno de desarrollo Spyder. Para la cosntrucción de este documento explicativo se utilizó el entorno de desarrollo Jupyter y formato Markdown. 

En el repositorio de github de este proyecto encontrarás, los insumos y el código y los productos del código aquí explicados. Esto para facilitar su reproductibilidad. **PONER LINK**



# Problema a resolver

Las tecnologías de la información nos permiten tener acceso a un gran número de noticias de distintos portales, y continuamente se están produciendo cada vez más noticias de manera acelerada. Por lo cual, el trabajo de una persona que se dedica a monitorear noticias se vuelve cada vez más complejo y exhaustivo. El flujo de trabajo de dicha persona es muy complejo, pues inicia identificando los criterios relevantes para su busqueda, sigue con la identificación de los sitios de noticias relevantes, y sigue con la lectura, análisis, clasificación y sistematización de cada una de las noticias que puedan ser relevantes. Sin embargo, nos podemos ayudar de los lenguajes de programación como Python y de su integració con Modelos grandes de lenguaje (LLM's, por sus siglas en inglés), para facilitar esta tarea. 

Así pues, el problema a resolver con esta demostración/proyecto, es el de simplificar el flujo de trabajo de un monitoreador de noticias, semi-automatizando ciertos procesos. Considero que es una semi-automatización debido a que considero que es importante dejar que en cada paso del proceso, el usuario tenga control para modificar las noticias que se toman en cuenta. Esto es muy importante, pues si bien un proyecto/demostración como este ayuda, no sustituye de ninguna manera el trabajo de un monitor experto, sólo pretende simplificarlo.

# Insumos 

Para este proyecto/demostración se presupone que el usario ya tiene identificado el tema de interés y los criterios de su busqueda y los portales de noticias que le interesan. Así pues, las herramientas necesarias son:

- Python.
- Algún LLM
- Excel

Y los insumos necesarios son:

- Páginas que se desean monitorear. En este caso: https://news.un.org/es/news


# Metodología

# Paso 1: Creamos el *scraper*

Como se presupone que ya se tiene identificada la página que queremos monitorear y los críterios de dicho monitoreo, el primer paso es crear un programa que nos permita recuperar la información relevante de la página de internet, es decir, un *scraper*. Para crear dicho programa, primero importamos las librerias pertinentes. 

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

Es importante mencionar que estas librerias funcionan para ciertas páginas, pues hay algunas que tienen una política anti-scraping, que impiden el uso de estas herramientas. Sin embargo, si el usuario se ve en esta situación, puede usar librerias alternas como Selene, o servicios como el que ofrece la página [Browse.AI](https://www.browse.ai/). Como por ahora sólo se trata de un demo, las librerías usadas son suficientes. 

Posteriormente, establecemos un header, que servirán como datos para *simular* una petición más humana a las páginas web que deseemos consultar. En este caso simulamos que usamos los navegadores específicados. Esto nos ayudará a evitar ciertos bloqueos. Así mismo, establecemos la página base a scrapear, y in timeout de 10 para evitar bloqueos en nuestro *scraper*

In [2]:
headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/58.0.3029.110 Safari/537.3"
    )
}

BASE_URL = "https://news.un.org/es/news?page="
TIMEOUT = 10

Posteriormente, establecemos la o las palabras clave que nos interesa buscar en el contenido de las noticias. También se pueden usar operadores como OR y AND para complejizar las palabras clave que deseamos. Posteiormente se crea una lista para guardar los resultados. 

In [4]:
# Lista de palabras clave a buscar en las noticia
keywords = ["inteligencia artificial", "Inteligencia Artificial"]

# Lista donde se guardarán los resultados del scraper
resultados_filtrados = []

Finalmente, creamos y ejecutamos nuestro *scraper*. La misión principal de este programa es recuperar el título, la url y la fecha de las noticias que cumplan los criterios. En este caso, que contenga alguna de las palabras clave y que hayan sido publicadas en el mes de mayo del 2025. 

Aquí hay varios puntos a notar:
- Se le da la indicación de buscar en las primeras 10 páginas del portal. Esto es así, porque sólo nos interesan las noticas de un mes en específico. En este sentido, una revisión prevía de la página base, nos revela que las noticias de un sólo mes, no abarcan más de 8 páginas. Aquí puse 10 para asegurar que se localicen todas las páginas deseadas. Como el mes de mi elección es mayo del 2025 (mes de creación de este demo), utilizo las primers 10 páginas.
- En cada proceso que puede fallar, en caso de que lo haga se configura para que salga un mensaje de error, esto ayuda a que en caso de qie se requieran ajustes, se pueda saber más fácilmente qué parte es la que falla. Los emojis son opcionales.
- Antes de correr el código, se tiene que explorar el código fuente de la página base para identificar los indicadores clave que nos interesan. En este caso, el contenedor donde se encuentra cada noticia, el contenedor del título, y el de la fecha de publicación. Cada página tiene sus propios indicadores, por lo cuál, el *scraper* debe ser adaptado a cada página que queramos monitorear.
- Se configura un mensaje para que al finalizar el programa veamos los resultados. En estricto sentido esto es opcional, pero es útil para seguir el paso del programa.

In [5]:
for page in range(10):  # Revisar páginas 0 a 9, pues las noticias del mes no están más allá de esta página. 
    url = f"{BASE_URL}{page}"
    print(f"Procesando: {url}")

    # Parte que descarga la página
    try:
        response = requests.get(url, headers=headers, timeout=TIMEOUT)
        response.raise_for_status()
    except requests.exceptions.Timeout:
        print(f"⚠️  La solicitud excedió el tiempo de espera en {url}")
        continue
    except requests.exceptions.RequestException as e:
        print(f"⚠️  Error al acceder a {url}: {e}")
        continue

    # Parte que estandariza o parsea el código html de las páginas. 
    soup = BeautifulSoup(response.text, "html.parser")

    # Parte que encuentra los contenedores de noticias <div class="views-row">
    noticias = soup.find_all("div", class_="views-row")
    for noticia in noticias:
        # Parte que extrae fechas desde <time class="datetime">
        time_element = noticia.find("time", class_="datetime")
        if not time_element:
            continue
        
        time_text = time_element.get_text(strip=True)
        datetime_attr = time_element.get("datetime", "")

        # Verificación de que sea del mes que queremos
        if not datetime_attr.startswith("2025-05"):
            continue

        # Parte que extraer título y enlace
        titulo_element = noticia.find(["h2", "h3"])
        if not titulo_element:
            continue
        
        enlace_element = titulo_element.find("a")
        if not enlace_element or not enlace_element.get("href"):
            continue

        titulo = enlace_element.get_text(strip=True)
        enlace_parcial = enlace_element["href"]
        if enlace_parcial.startswith("http"):
            enlace = enlace_parcial
        else:
            enlace = "https://news.un.org" + enlace_parcial

        # Parte que hace una segunda solicitud para verificar el contenido completo y ver si contiene las palabras clave
        try:
            resp_noticia = requests.get(enlace, headers=headers, timeout=TIMEOUT)
            resp_noticia.raise_for_status()
        except requests.exceptions.Timeout:
            print(f"⚠️  Timeout al obtener contenido de la noticia: {enlace}")
            continue
        except requests.exceptions.RequestException as e:
            print(f"⚠️  Error al obtener contenido de la noticia en {enlace}: {e}")
            continue

        # Parte para obtener el texto completo y buscar cualquiera de las palabras clave
        soup_noticia = BeautifulSoup(resp_noticia.text, "html.parser")
        contenido = soup_noticia.get_text(separator=" ", strip=True).lower()
        
        #Parte para verifciar si hay palabras las palabras clave
        if any(keyword in contenido for keyword in keywords):
            resultados_filtrados.append({
                "titulo": titulo,
                "url": enlace,
                "fecha": time_text
            })
#nota: Si quieremos que aparezcan todas las palabras, cambiar el any por all

# Parte para mostrar las noticias filtradas en la consola
print("\n=== RESULTADOS (contienen al menos una de las keywords) ===")
for i, item in enumerate(resultados_filtrados, start=1):
    print(f"{i}. [{item['fecha']}] {item['titulo']}")
    print(f"   URL: {item['url']}\n")

print(f"Total de noticias encontradas: {len(resultados_filtrados)}")

Procesando: https://news.un.org/es/news?page=0
Procesando: https://news.un.org/es/news?page=1
Procesando: https://news.un.org/es/news?page=2
Procesando: https://news.un.org/es/news?page=3
Procesando: https://news.un.org/es/news?page=4
Procesando: https://news.un.org/es/news?page=5
Procesando: https://news.un.org/es/news?page=6
Procesando: https://news.un.org/es/news?page=7
Procesando: https://news.un.org/es/news?page=8
Procesando: https://news.un.org/es/news?page=9

=== RESULTADOS (contienen al menos una de las keywords) ===
1. [12 Mayo 2025] El cambio climático afecta cada vez más a los países africanos
   URL: https://news.un.org/es/story/2025/05/1538596

2. [6 Mayo 2025] Ayuda a Gaza, refugiados de Sudán, desarrollo humano... Las noticias del martes
   URL: https://news.un.org/es/story/2025/05/1538536

3. [6 Mayo 2025] Desaceleración «alarmante» del desarrollo humano: ¿podría la IA aportar respuestas?
   URL: https://news.un.org/es/story/2025/05/1538516

4. [2 Mayo 2025] Gaza, refug

Finalmente, para llevar a cabo futuros proceso, convertimos los resultados en una base de datos. 

In [6]:
base_noticias = pd.DataFrame(resultados_filtrados)
print(base_noticias)

                                              titulo  \
0  El cambio climático afecta cada vez más a los ...   
1  Ayuda a Gaza, refugiados de Sudán, desarrollo ...   
2  Desaceleración «alarmante» del desarrollo huma...   
3  Gaza, refugiados, libertad de prensa... Las no...   
4  Los periodistas de Gaza son testigos y víctima...   

                                            url         fecha  
0  https://news.un.org/es/story/2025/05/1538596  12 Mayo 2025  
1  https://news.un.org/es/story/2025/05/1538536   6 Mayo 2025  
2  https://news.un.org/es/story/2025/05/1538516   6 Mayo 2025  
3  https://news.un.org/es/story/2025/05/1538451   2 Mayo 2025  
4  https://news.un.org/es/story/2025/05/1538436   2 Mayo 2025  


De quererlo, guardamos los resultados en un archivo excel. Para lograrlo, sólo establecemos una ruta de salida donde se guardará el archivo, y ejecutamos el método (función) correspondiente. Notece que la base contiene tres columnas, cada una con los datos que se querían recuperar  (título, url, fecha de publicación). Esto es importante, pues permitirá al usuario verificar manualmente cada una de las noticias arrojadas por el *scraper*, y determinar su relevancia. 

In [ ]:
ruta_sal =r"C:\Users\javal\OneDrive\Desktop\Portafolio 9\04_demo_monitoreo_noticias IA\04\base_noticias.xlsx"
base_noticias.to_excel(ruta_sal, index=False)

# Paso 2: Clasificar noticias con IA.

Una vez que el usaurio haya verificado las noticias de la lista, el siguiente paso es clasificar las noticias entre las que son relevantes para nuestros propósitos de las que no. Tradicionalmente, esto tendríamos que hacer leyendo y analizando una por una las noticias. Y si bien aquí sólo son cinco noticias, hay ocasiones en las que pueden ser muchísimas. Sin embargo, podemos ayudarnos de los LLM's para facilitar este trabajo. 

Así pues, primero cargamos nuestra base de noticias (si es que se modificó la base del paso anterior), y hacemos una lista con las url de las noticias, que ya tenemos.

In [9]:
base_noticias_pre = pd.read_excel(r"C:\Users\javal\OneDrive\Desktop\Portafolio 9\04_demo_monitoreo_noticias IA\04\base_noticias.xlsx")

urls = base_noticias_pre ['url'].dropna().tolist() 

print (base_noticias_pre)



                                              titulo  \
0  El cambio climático afecta cada vez más a los ...   
1  Ayuda a Gaza, refugiados de Sudán, desarrollo ...   
2  Desaceleración «alarmante» del desarrollo huma...   
3  Gaza, refugiados, libertad de prensa... Las no...   
4  Los periodistas de Gaza son testigos y víctima...   

                                            url         fecha  
0  https://news.un.org/es/story/2025/05/1538596  12 Mayo 2025  
1  https://news.un.org/es/story/2025/05/1538536   6 Mayo 2025  
2  https://news.un.org/es/story/2025/05/1538516   6 Mayo 2025  
3  https://news.un.org/es/story/2025/05/1538451   2 Mayo 2025  
4  https://news.un.org/es/story/2025/05/1538436   2 Mayo 2025  


In [10]:
print (urls)

['https://news.un.org/es/story/2025/05/1538596', 'https://news.un.org/es/story/2025/05/1538536', 'https://news.un.org/es/story/2025/05/1538516', 'https://news.un.org/es/story/2025/05/1538451', 'https://news.un.org/es/story/2025/05/1538436']


Posteriormente, hacemos otro *scraper* que recupere el contenido de cada una de las url de la lista. Para ello, usamos una librería especializada en la tarea *newspaper*. Guardamos el contenido de cada noticia en una lista creada antes de ejecutar. 

In [14]:
from newspaper import Article
import time

contenidos_pre = []

for url in urls:
    try:
        articulo = Article(url)
        articulo.download()
        articulo.parse()
        contenidos_pre.append(articulo.text)
    except Exception as e:
        print(f"❌ Error al procesar {url}: {e}")
        contenidos_pre.append('Error')

    # Pausa de 1.5 segundos entre cada descarga
    time.sleep(1.5)

La lista de los contenidos de cada noticia luciría así:

In [15]:
print(contenidos_pre)

['Las condiciones meteorológicas extremas y las consecuencias del cambio climático están afectando el desarrollo socioeconómico de África: exacerbando el hambre, la inseguridad y los desplazamientos, según el informe publicado por la Organización Meteorológica Mundial (OMM) este lunes sobre el estado del clima en África 2024.\n\nEl informe destaca los retos para la agricultura y el medio ambiente; la seguridad alimentaria, hídrica y energética; y la salud y la educación, haciendo hincapié en las oportunidades y las nuevas herramientas para hacer frente a los retos.\n\n“El informe sobre el estado del clima en África refleja la realidad urgente y creciente del cambio climático en todo el continente”, afirmó Celeste Saulo, secretaria general de la OMM.\n\n“También revela un patrón marcado de fenómenos meteorológicos extremos, en el que algunos países se enfrentan a inundaciones excepcionales causadas por precipitaciones excesivas, y otros soportan sequías persistentes y escasez de agua”.\

Ahora, creamos un módulo que nos permita análizar y clasificar cada uno de los contenidos de la lista. Por vuestiones de privacidad, no puedo mostrar los datos del LLM que usé, basta decir que es uno muy popular. Además, no importa cual LLM se utilice. Con un *prompt* adecuado los resultados pueden ser suficientes. Así pues, del código omitiré las referencias al LLM usado, baste decir que se tiene que importar la o las librerías adecuadas y conectarse al LLM con un API (o de manera local si se tiene descargado el modelo).

In [16]:
 clasificaciones = []

Una vez que se carga y se conecta con el LLM, se crea una lista vacía en donde irán las clasificaciones de las noticias hechas por el modelo. Posteriormente se itera sobre cada uno de los elementos de la lista de contenidos, la instrucción que se le da al LLM, y cada respuesta se guarda en la lista de clasificaciones. 

Para que la respuesta del LLM sea lo más adecuada posible, se tienen que crear un *propmt* adecuado, indicando el tipo de contenido que se dará, y el tipo de respuesta que deseamos obtener, así como algunos ejemplos. También es útil asignar roles al LLM para que de la respuesta con cierto estilo. En este punto se pueden utilizar diferentes técnicas de *prompt engineering*. Depende del usuario experimentar y ajustar su *prompt*. En est caso, se utilizó un *prompt* simple, pero funcional. Para consultar técnicas de *prompt engineering*: <a href="https://www.promptingguide.ai/es" target="_blank">prompt engineering guide</a>.

A continuación dejó una imagen del prompt que utilicé para la clasificación:

<div style="text-align: center;">
  <p><strong>Imagen 1: Prompt para clasificar</strong></p>
  <img src="imagenes/01_prompt_clasificar.png" width="70%">
  <p><em>Fuente: Elaboración propia</em></p>
</div>

Notece, que en este caso, hice que clasificara las noticias identifcando si hablaban sobre Gaza o algo relacionado con dicho lugar. Elegí esta orden para fines de esta demostración, pues anteriormente habia identificado que había algunas noticias que hablaban sobre este lugar, así que sería fácil corrorborar los resultados. Naturalmente, el usuario puede pedir que clasifique en función de cualquier otro criterio, como que se mencione cierta entidad o no, o que se trate sobre un cierto tema o no.

Además, pedí que se diera una razón para justificar la clasificiación, lo que es útil para el usuario, pues le daría una guía para hacer su propia clasificación. Finalmente, agregamos la lista de las clasificaciones como una nueva columna a la base de datos de las noticias. Como la lista de contenidos y clasificacines tienen el mismo orden que las observaciones del base de noticias, pues salieron de allí, no hay problemas de orden. Así, pues, la nueva base con las clasificaciones quedaría algo como lo siguiente: 

In [19]:
base_noticias_pre['clasificacion'] = clasificaciones

print (base_noticias_pre)

                                              titulo  \
0  El cambio climático afecta cada vez más a los ...   
1  Ayuda a Gaza, refugiados de Sudán, desarrollo ...   
2  Desaceleración «alarmante» del desarrollo huma...   
3  Gaza, refugiados, libertad de prensa... Las no...   
4  Los periodistas de Gaza son testigos y víctima...   

                                            url         fecha  \
0  https://news.un.org/es/story/2025/05/1538596  12 Mayo 2025   
1  https://news.un.org/es/story/2025/05/1538536   6 Mayo 2025   
2  https://news.un.org/es/story/2025/05/1538516   6 Mayo 2025   
3  https://news.un.org/es/story/2025/05/1538451   2 Mayo 2025   
4  https://news.un.org/es/story/2025/05/1538436   2 Mayo 2025   

                                       clasificacion  
0             Negativo, el texto no menciona a Gaza.  
1  afirmativo, el texto menciona a Gaza ya que se...  
2             negativo, el texto no menciona a Gaza.  
3  afirmativo, el texto menciona a Gaza explícita...

Filtramos las noticias que no son relevantes, para quedarnos con una nueva base con las noticias de interés. 

In [24]:
base_def_noticias = base_noticias_pre[~base_noticias_pre['clasificacion'].isin( ['Negativo, el texto no menciona a Gaza.', 'negativo, el texto no menciona a Gaza.'])]
print (base_def_noticias)

                                              titulo  \
1  Ayuda a Gaza, refugiados de Sudán, desarrollo ...   
3  Gaza, refugiados, libertad de prensa... Las no...   
4  Los periodistas de Gaza son testigos y víctima...   

                                            url        fecha  \
1  https://news.un.org/es/story/2025/05/1538536  6 Mayo 2025   
3  https://news.un.org/es/story/2025/05/1538451  2 Mayo 2025   
4  https://news.un.org/es/story/2025/05/1538436  2 Mayo 2025   

                                       clasificacion  
1  afirmativo, el texto menciona a Gaza ya que se...  
3  afirmativo, el texto menciona a Gaza explícita...  
4  Afirmativo, el texto menciona explícitamente a...  


Finalmente, exportamos la base de datos como un archivo excel, en caso de que el usuario desee manipular o verificar dicha base.

In [25]:
ruta_sal_def =r"C:\Users\javal\OneDrive\Desktop\Portafolio 9\04_demo_monitoreo_noticias IA\04\base_noticias_def.xlsx"
base_def_noticias.to_excel(ruta_sal_def, index=False)

El archivo de excel quedaría algo como lo siguiente:

<div style="text-align: center;">
  <p><strong>Imagen 2: Archivo excel con las clasificaciones propuestas por el LLM</strong></p>
  <img src="imagenes/02_clasificacion_excel.png" width="70%">
  <p><em>Fuente: Elaboración propia</em></p>
</div>

# Paso 3: Resumir noticias con IA

Una vez que se el usuario ha decidido de manera definitiva las noticias que son relevantes, quizá tenga que hacer un reporte de dichas noticias, y quizá dicho reporte deberá contener un resumen con ciertas características. Para ello, también nos podemos apoyar de la IA. Para ello, utilizaremos el mismo LLM del paso anterior (o bien se puede usar otro que sea especializado en la tarea). 

Com presuponemos que el usario hizo modificaciones al excel del paso anterior, cargamos la base modificada, le quitamos los valores nulos, y hacemos una lista de las url's, al igual que en el paso anterior. Nos saldrá algo como lo siguiente: 

In [26]:
base_resumenes = pd.read_excel(r"C:\Users\javal\OneDrive\Desktop\Portafolio 9\04_demo_monitoreo_noticias IA\04\base_noticias_def.xlsx")

urls_resumenes = base_resumenes['url'].dropna().tolist()

In [27]:
print (base_resumenes)

                                              titulo  \
0  Ayuda a Gaza, refugiados de Sudán, desarrollo ...   
1  Gaza, refugiados, libertad de prensa... Las no...   
2  Los periodistas de Gaza son testigos y víctima...   

                                            url        fecha  \
0  https://news.un.org/es/story/2025/05/1538536  6 Mayo 2025   
1  https://news.un.org/es/story/2025/05/1538451  2 Mayo 2025   
2  https://news.un.org/es/story/2025/05/1538436  2 Mayo 2025   

                                       clasificacion  
0  afirmativo, el texto menciona a Gaza ya que se...  
1  afirmativo, el texto menciona a Gaza explícita...  
2  Afirmativo, el texto menciona explícitamente a...  


In [28]:
print (urls_resumenes)

['https://news.un.org/es/story/2025/05/1538536', 'https://news.un.org/es/story/2025/05/1538451', 'https://news.un.org/es/story/2025/05/1538436']


Al igual que en el paso anterior, hacemos un *scraper* pararecuperar los contenidos de las noticias, y guardamos dichos contenidos en una lista: 

In [29]:
contenidos_resumenes = []

for url in urls_resumenes:
    try:
        articulo = Article(url)
        articulo.download()
        articulo.parse()
        contenidos_resumenes.append(articulo.text)
    except Exception as e:
        print(f"❌ Error al procesar {url}: {e}")
        contenidos_resumenes.append('Error')

    # Pausa de 1.5 segundos entre cada descarga
    time.sleep(1.5)

Al igual que en paso anterior, se crea una lista para guardar los resumenes generados por el LLM.

In [30]:
resumenes = []

Naturalmente, a diferencia del paso anterior, aquí modificamos el *prompt* para que nos de un resumen. Para hacer obvio la influencia de la indicación, pedí que me contará el resumen como si fuera un cuento de hadas. El *propmt* utilizado fue el siguiente:

<div style="text-align: center;">
  <p><strong>Imagen 3: Prompt para resumir</strong></p>
  <img src="imagenes/03_prompt_resumir.png" width="70%">
  <p><em>Fuente: Elaboración propia</em></p>
</div>

Finalmente, anexamos los resumenes a la base que teníamos, y nos quedaría algo como lo siguiente:

In [31]:
base_resumenes['resumen'] = resumenes
print (base_resumenes)

                                              titulo  \
0  Ayuda a Gaza, refugiados de Sudán, desarrollo ...   
1  Gaza, refugiados, libertad de prensa... Las no...   
2  Los periodistas de Gaza son testigos y víctima...   

                                            url        fecha  \
0  https://news.un.org/es/story/2025/05/1538536  6 Mayo 2025   
1  https://news.un.org/es/story/2025/05/1538451  2 Mayo 2025   
2  https://news.un.org/es/story/2025/05/1538436  2 Mayo 2025   

                                       clasificacion  \
0  afirmativo, el texto menciona a Gaza ya que se...   
1  afirmativo, el texto menciona a Gaza explícita...   
2  Afirmativo, el texto menciona explícitamente a...   

                                             resumen  
0  Había una vez un lugar llamado Gaza, donde la ...  
1  Había una vez en Gaza, donde la gente sufría s...  
2  Había una vez un valiente reportero llamado Sa...  


Finalmente, se puede guardar esta base en un archivo excel, para futuros procesos o consultas.

In [32]:
ruta_sal_con_resumen =r"C:\Users\javal\OneDrive\Desktop\Portafolio 9\04_demo_monitoreo_noticias IA\04\base_noticias_def_resumen.xlsx"
base_resumenes.to_excel(ruta_sal_con_resumen, index=False)

<div style="text-align: center;">
  <p><strong>Imagen 4: Archivo excel con resumenes</strong></p>
  <img src="imagenes/04_resumenes_excel.png" width="70%">
  <p><em>Fuente: Elaboración propia</em></p>
</div>

En la siguiente imagen se puede apreciar una sola línea para ver el resumen en detalle.

<div style="text-align: center;">
  <p><strong>Imagen 5: Archivo excel con resumenes, detalle de una sola observación</strong></p>
  <img src="imagenes/05_resumen_detalle.png" width="70%">
  <p><em>Fuente: Elaboración propia</em></p>
</div>

# Paso 4: Reporte de resultados.

A partir de los insumos producidos el usuario ya podría hacer un reporte con los insumos necesarios obtenidos en este punto. Sin embargo, si tenemos una plantilla de word con la estructura de los reportes, podríamos facilitar un poco el proceso. (Para saber más de cómo llenar documentos/plantillas con datos de manera automática ver: <a href="https://alevillegas9.github.io/02_Llenado_automatico_documentos/02.html" target="_blank">02. Llenado automático de archivos desde una base de datos con python</a> )

Así pues, importamos la librería que nos permite manipular documentos de word, y cargamos nuestra plantila. Recuerda que la plantilla debe de tener adecuadamente indicados los marcadores de los datos a llenar, y que para facilitar el proceso los marcadores deben coíncidiro con el nombre de las columnas de donde se tomarán los datos.

<div style="text-align: center;">
  <p><strong>Imagen 6: Ejemplo de plantilla para reporte </strong></p>
  <img src="imagenes/06_plantilla.png" width="70%">
  <p><em>Fuente: Elaboración propia</em></p>
</div>

In [33]:
from docx import Document

ruta_plantilla = r"C:\Users\javal\OneDrive\Desktop\Portafolio 9\04_demo_monitoreo_noticias IA\04\noticias_relevantes_plantilla.docx"

plantilla = Document(ruta_plantilla)


Después, transformamos la plantilla en un string, para su manipulación. 

In [35]:
plantilla_cont = "\n".join([p.text for p in plantilla.paragraphs])
print (plantilla_cont)

Noticia relevante
Título: {{titulo}}
Link: {{url}}
Fecha de publicación: {{fecha}}
Resumen: {{resumen}}


Creamos un reporte que es un documento word vacio al principio, y después lo llenamos con una iteración sobre cada observación de la base de datos. Notece la linea 4 del ciclo for, donde se da la instrucción de qué dato debe llenar qué marcador. 

In [36]:
reporte = Document()

for _, fila in base_resumenes.iterrows():
    texto = plantilla_cont
    for clave, valor in fila.items():
        marcador = f'{{{{{clave}}}}}'
        texto = texto.replace(marcador, str(valor))
    for linea in texto.split('\n'):
        reporte.add_paragraph(linea)
    reporte.add_paragraph("\n")

Guardamos documeto

In [ ]:
reporte.save(r"C:\Users\javal\OneDrive\Desktop\Portafolio 9\04_demo_monitoreo_noticias IA\04\reporte.docx")

Y nos saldría un documento como el siguiente:

<div style="text-align: center;">
  <p><strong>Imagen 7: Ejemplo de reporte automatizado </strong></p>
  <img src="imagenes/07_reporte.png" width="70%">
  <p><em>Fuente: Elaboración propia</em></p>
</div>

Finalmente, el usuario puede modificar el reporte, o modificar la plantilla para que la modificación del reporte no sea necesaria.

# Conclusiones

El proceso de monitoreo de noticias puede ser muy exhaustivo, sobretodo cuando se tienen que monitorear una gran cantidad de portales, de noticias individuales y en un periodo corto de tiempo. Sin embargo, gracias a las nuevas tecnologías de IA y herramientas como python, este proceso se puede simplificar de manera considerable, sin sustituir la acción humana. En este sentido, en este documento se mostró una pequeña demostración de como esto puede ser posible, y cómo en cada paso el usuario puede tener el control del proceso y supervisarlo. 

Estoy convencido de que las herramientas de IA no podrán sustituir a la acción y conocimiento de un experto, pero sí puede ser una herramienta muy valiosa para potenciar su accción. 